In [ ]:
#accessing netcdf files from Hector's folders
import os, glob, sys
sys.path.append("//nobackup//amondal//Python//Hector_Python_Scripts")
sys.path.append("//nobackup/amondal//Python//mit_geos_analysis")
#from mds_store import openmdsdataset
#playing with netcdf - xarray
import numpy as np
import xarray as xr
import dask.array as da
import matplotlib.pyplot as plt
# How about visualization?
%matplotlib inline
import netCDF4
from netCDF4 import Dataset
#from xmitgcm import open_mdsdataset
from GetNCDataSet import getMITNCDataSet
from MITllc2160Depth import *
sys.path.append("//nobackup/amondal//Python//xmitgcm//xmitgcm")
from xmitgcm.mds_store import open_mdsdataset
import xrft

from dask.distributed import Client
client = Client(memory_limit='90GB',n_workers = 40, threads_per_worker=1)

------------------------------------------------------------------------------------------------------------------
Below we're calculating and plotting spectra - this is hector's code - I will leave this notebook as an example of a successful spectra calculation. There could be work done to make this faster. Probably. But it may not be necessary.

In [86]:
W15 = xr.load_dataset('/nobackup/amondal/NCData/W15_reg_for_Hector.nc4')

In [88]:
W15 = W15['__xarray_dataarray_variable__']

In [89]:
W15

<xarray.DataArray '__xarray_dataarray_variable__' (time: 2208, xdim: 300, ydim: 375)>
array([[[-1.79072056e-04, -1.79072056e-04,  1.40419088e-05, ...,
          6.94546849e-04, -5.72377467e-05,  1.71237916e-04],
        [-1.90685940e-04, -1.90685940e-04,  3.65297383e-05, ...,
          3.92997375e-04, -4.30897664e-04, -2.05140750e-05],
        [ 2.97633414e-05,  2.97633414e-05,  1.76064001e-04, ...,
          1.32667337e-04, -6.93743466e-04, -2.09442544e-04],
        ...,
        [-8.97452555e-05, -8.97452555e-05,  2.82943627e-04, ...,
          6.29509814e-05, -5.04362069e-05, -2.01856703e-04],
        [ 2.58425571e-05,  2.58425571e-05,  1.38801173e-04, ...,
         -2.74639227e-04, -2.47196149e-04, -2.91474396e-04],
        [ 3.76986418e-06,  3.76986418e-06, -2.98216310e-05, ...,
         -1.52595079e-04, -6.50198053e-05, -5.23657873e-05]],

       [[ 4.18512936e-05,  4.18512936e-05,  2.45252886e-04, ...,
          7.44183664e-04,  5.82686989e-05,  2.70126970e-04],
        [-9.00370796e-05, -9.00370796e-05,  1.21546596e-04, ...,
          9.62953054e-05, -7.52933905e-04, -1.23979684e-04],
        [-1.97881527e-05, -1.97881527e-05,  1.11717061e-04, ...,
         -1.73661450e-04, -1.10437348e-03, -4.10272798e-04],
...
        [ 5.95370482e-04,  5.95370482e-04, -2.74199876e-04, ...,
         -6.54706237e-05,  1.26383704e-04,  3.47939553e-04],
        [ 5.00473485e-04,  5.00473485e-04, -4.01498808e-04, ...,
         -1.49221451e-04, -1.47778046e-04, -5.04979562e-06],
        [ 3.30017385e-04,  3.30017385e-04, -4.68386686e-04, ...,
          7.59719987e-05, -1.45967442e-06, -5.16377077e-05]],

       [[ 2.92079465e-04,  2.92079465e-04,  4.65510064e-04, ...,
          1.02395122e-03, -1.95911503e-04, -6.83746533e-04],
        [ 3.06014204e-04,  3.06014204e-04,  5.19809430e-04, ...,
          6.38214755e-04, -3.69670888e-04, -4.67681733e-04],
        [-9.05534107e-05, -9.05534107e-05,  3.99219483e-04, ...,
          6.80924422e-05, -3.12382297e-04, -9.97251336e-05],
        ...,
        [ 2.49944918e-04,  2.49944918e-04, -8.95821358e-05, ...,
          2.60957186e-05, -1.92118023e-05,  6.48331770e-05],
        [ 3.89522756e-04,  3.89522756e-04, -1.39907686e-04, ...,
          3.02137109e-04,  5.16373366e-05, -7.44520294e-05],
        [ 5.06728771e-04,  5.06728771e-04, -1.79205555e-04, ...,
          3.41602397e-04,  3.26617737e-04,  5.38164095e-05]]])
Coordinates:
  * time     (time) float64 0.0 1.0 2.0 3.0 ... 2.205e+03 2.206e+03 2.207e+03
  * xdim     (xdim) float64 0.0 2.224 4.448 6.672 ... 658.3 660.5 662.7 665.0
  * ydim     (ydim) float64 0.0 4.448 8.896 ... 1.655e+03 1.659e+03 1.664e+03

In [1]:
W15_np = W15.values
W15_np_swap = np.swapaxes(np.swapaxes(W15_np, 0, 1), 1,2)

#hector's code assumes a certain ordering of the dimensions (x,y,time) - the xarray code always goes time first

NameError: name 'W15' is not defined

In [113]:
W15_np_swap.shape

(300, 375, 2208)

In [92]:
dx = W15.xdim.diff('xdim').mean().values
dy = W15.ydim.diff('ydim').mean().values
dt = W15.time.diff('time').mean().values

In [116]:
dt

array(1.)

In [96]:
# detrend: space and time
from scipy import signal
u = W15_np_swap
u = signal.detrend(u,axis=0,type='linear')
u = signal.detrend(u,axis=1,type='linear')
u = signal.detrend(u,axis=2,type='linear')
print('------- End detrending --------')

------- End detrending --------


In [107]:
2208/30

73.6

In [147]:
# Preambule
iy,ix,it = u.shape
#print(it,iy,ix)
#this implements bartlett's method - you can also try changing this to Welch's for even better resolution
chunksize_in_days = 30 #allows you to pick your chunk size
iaux = (chunksize_in_days*24)#(60*24*7)/10 #(7*24)
nt = np.around(it/(iaux),decimals=1)
print(nt)

# Calclate the 3D spectrum
for i in range(int(nt)):
      uaux = u[:,:,i*iaux:i*iaux+iaux]
      if i == 0:
         Eu,k,l,om = wf_spectrum.spec_est3(uaux,dy,dy,dt)
         #notice we assume that the grid is regular in x and y - use the bigger interval to more accurately reflect your resolution
      else:
         Eua,_,_,_ = wf_spectrum.spec_est3(uaux,dy,dy,1)
         Eu = Eu + Eua
Eu = Eu/nt
#USE SAME RESOLUTION IN THE DX AND DY!!!

3.1


In [148]:
def calc_ispec(k,l,E):
    """ calculates isotropic spectrum from 2D spectrum """

    dk,dl = k[1,]-k[0],l[1]-l[0]
    l,k = np.meshgrid(l,k)
    wv = np.sqrt(k**2 + l**2)

    if k.max()>l.max():
        kmax = l.max()
    else:
        kmax = k.max()

    # create radial wavenumber
    dkr = np.sqrt(dk**2 + dl**2)
    kr =  np.arange(dkr/2.,kmax+dkr,dkr)
    ispec = np.zeros(kr.size)
    #print(ispec.shape)
    #print(kr.shape)
    for i in range(kr.size):
        fkr =  (wv>=kr[i]-dkr/2) & (wv<=kr[i]+dkr/2)
    #    print(fkr.shape)
        dth = np.pi / (fkr.sum()-1)
        ispec[i] = E[fkr].sum() * kr[i] * dth

    return kr, ispec

In [149]:
I = 0
for i in range(om.size-1):
    kiso,Ei = calc_ispec(k,l,Eu[:,:,i])
    if I == 0:
        Eiso = np.empty((len(Ei),om.size))
    Eiso[:,i] = Ei
    I = I + 1

In [2]:
#isotropize is good code but if everything else is in numpy you may as well do this in numpy too

In [150]:
np.isnan(Eiso).sum()

0

In [80]:
#Eiso_units= Eiso#*4.2e6
#Eiso_units.shape

(70, 626)

The "60" in the calculation is effectively the Window correction. Apparently when you use a Hanning window and calculate the window correction, for the spectrum it should always be about 60

In [ ]:
#====================================================
#===================================================
kk = kiso[1:]
omm = om[1:]
#ff = coriolis(38)
#f0 =  0.89e-4
#print(f0)
#kiso = np.logspace(-3,0.,100)

#################
fig = plt.figure(figsize=(6,5))

#fig = plt.figure(figsize=(10,6))
#ax1 = plt.subplot2grid((3,6),(0,1),colspan=2)
#ax2 = plt.subplot2grid((3,6),(1,0),rowspan=2)
#ax3 = plt.subplot2grid((3,6),(1,1),rowspan=2, colspan=2)
ax1 = fig.add_subplot(111)
cs=plt.pcolormesh(kk,omm,60*1*Eiso.T[1:,1:]*kk[None,...]*omm[...,None],
                   shading='gouraud',cmap='rainbow',norm=LogNorm() )
ax1.set_yscale('log')
ax1.set_xscale('log')
plt.title(r'Vertical Velocity (W) Spectrum (45m) (March 1 - April 20)', size = 16)
plt.xlabel(r'Wavenumber[cpkm]', size=16)
plt.ylabel(r'Frequency [cph]', size = 16)
#plt.xlabel(r'$\kappa$ [cpkm]',size=16)
#plt.ylabel(r'Frequency [cph]',size=16)
plt.clim([1e-8,1e-6])
plt.colorbar()
#ax1.set_ylim([omm[1], omm[-1]])
#ax1.set_xlim([kk[1],1e-1])
#ax3.set_yticks([])
#plt.title('Mode-1',fontweight='bold',size=20)
#

#plt.savefig(PrntOut+figurename+output_format,format='png',
#            dpi=500,bbox_inches='tight')

plt.show()


Below gives an example of how to plot with km ticks and time ticks acros from the frequency wavenumber ones

In [ ]:
fig = plt.figure(figsize=(12,4))
plt.subplots_adjust(wspace=0.4)


ax1=fig.add_subplot(131)
cs=plt.pcolormesh(kiso_ecm,om_ecm,
                  (Eiso_ecm.T*kiso_ecm[None,...]*om_ecm[...,None]),
                  vmin=1e-2,vmax=5e0,cmap='rainbow',norm=LogNorm())
ax1.set_yscale('log')
ax1.set_ylim(1/(24.*40),1/2.)
ax1.set_xscale('log')
ax1.set_xlim(1./1000.,1/10.)
plt.ylabel(r'Frequency [cph]',size=15,fontweight='normal')
plt.xlabel('Wavenumber [cpkm]',size=15)
ax1a = ax1.twiny()
ax1a.set_yscale('log')
ax1a.set_xscale('log')
ax1a.set_xlim(1./1000.,1/10.)
ax1a.set_xticks([1/1000.,1/100.,1/10.])
ax1a.set_xticklabels(['1000','100','10'])
ax1a.set_xlabel('Wavelength [km]',size=15)
ax1a = ax1.twinx()
ax1a.set_yscale('log')
ax1a.set_xscale('log')
ax1a.set_ylim(1/(24.*40.),1/2.)
ax1a.set_yticks([1/24.,1/(24.*7.),1/(24*30.)])
ax1a.set_yticklabels(['1','7','30'])
plt.text(1/50.,1/(5.),r'ECMWF',
         size=12,fontweight='bold',ha="center",va="center",
         bbox=dict(boxstyle="round",
                   ec='k',
                   fc='w',
                   ))




ax1=fig.add_subplot(132)
cs=plt.pcolormesh(kiso_era,om_era,
                  (Eiso_era.T*kiso_era[None,...]*om_era[...,None]),
                  vmin=1e-2,vmax=5e0,cmap='rainbow',norm=LogNorm())
ax1.set_yscale('log')
ax1.set_ylim(1/(24.*40),1/2.)
ax1.set_xscale('log')
ax1.set_xlim(1./1000.,1/10.)
plt.xlabel('Wavenumber [cpkm]',size=15)
ax1a = ax1.twiny()
ax1a.set_yscale('log')
ax1a.set_xscale('log')
ax1a.set_xlim(1./1000.,1/10.)
ax1a.set_xticks([1/1000.,1/100.,1/10.])
ax1a.set_xticklabels(['1000','100','10'])
ax1a.set_xlabel('Wavelength [km]',size=15)
ax1a = ax1.twinx()
ax1a.set_yscale('log')
ax1a.set_xscale('log')
ax1a.set_ylim(1/(24.*40.),1/2.)
ax1a.set_yticks([1/24.,1/(24.*7.),1/(24*30.)])
ax1a.set_yticklabels(['1','7','30'])